In [1]:
# import libraries
from gensim.models.word2vec import Word2Vec
import gensim
from gensim.models.doc2vec import TaggedDocument
import multiprocessing
from sklearn import utils
import pandas as pd
from gensim.models.phrases import Phrases, Phraser
import numpy as np
import xgboost 
from tqdm import tqdm
from sklearn.model_selection import train_test_split
tqdm.pandas(desc="progress-bar")
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn import decomposition, ensemble
from collections import Counter
from nltk import ngrams

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
def classification_report(x_train, x_test, y_train, y_test):
  models = []
  models.append(('LogisticRegression', linear_model.LogisticRegression(solver='newton-cg',multi_class='multinomial')))
  models.append(('RandomForest', ensemble.RandomForestClassifier(n_estimators=100)))

  for name, model in models:
      clf=model
      clf.fit(x_train, y_train)
      y_pred=clf.predict(x_test)
      print(f"{name}:")
      print(f"accuracy: {metrics.accuracy_score(y_pred=y_pred, y_true=y_test)}")
      print(f"precision: {metrics.precision_score(y_pred=y_pred, y_true=y_test, average='macro')}")
      print(f"recall: {metrics.recall_score(y_pred=y_pred, y_true=y_test, average='macro')}")
      print(f"{metrics.classification_report(y_pred=y_pred, y_true=y_test)}")

In [3]:
def get_word_counts(data):
  words = data.tweet.to_string().split()
  return Counter(words)
  

In [4]:
# word2vec hyper parameters
# min_count = corpusta kelimenin en az bulunma sayısı eğer kelime bu kadardan az geçiyorsa anlam ifade etmediği varsayılır(default=5)
# vector_size = kelimelerin ifade edileceği vektörün boyut sayısı
# window = current ve predicted word arasındaki maksimum mesafe
# sg = 0 cbow, 1 skip-gram
# negative = eğer sıfırdan büyük olursa negative sampling kullanılır 5-20 arasında olmalı
# alpha = başlangıç learning rate
# min_alpha = eğitim aşamasında learning rate linear olarak bunu düşer
# epoch = iterasyon sayısı


In [5]:
def labelize_tweets_ug(tweets,label):
    result = []
    prefix = label
    for i, t in zip(tweets.index, tweets):
        result.append(TaggedDocument(t.split(), [prefix + '_%s' % i]))
    return result

In [6]:
def train_word2vec(corpus, n_epoch, name_corpus, sg, vector_size, negative, window, min_count, alpha, min_alpha):
  cores = multiprocessing.cpu_count()
  model = Word2Vec(sg=sg, size=vector_size, negative=negative, window=window, min_count=min_count, workers=cores, alpha=alpha, min_alpha=min_alpha)
  model.build_vocab([x.words for x in tqdm(corpus)])

  for epoch in range(n_epoch):
    model.train(utils.shuffle([x.words for x in tqdm(corpus)]), total_examples=len(corpus), epochs=1)
    model.alpha -= 0.002
    model.min_alpha = model.alpha

  model.save(f"/content/drive/MyDrive/hesaplamalı_anlambilim_ödev/trained_embeddings/{name_corpus}_sg_{sg}_size_{vector_size}_window_{window}_min_count_{min_count}.model")
  return model

In [7]:
def get_mean_vector(model, words):
    # remove out-of-vocabulary words
    words = [word for word in words if word in model.wv]
    if len(words) >= 1:
        return np.mean(model[words], axis=0)
    else:
        return np.zeros((1, model.vector_size))

In [8]:
def get_vectors(model, corpus):
  vectors = []
  for sentence in corpus:
      vec = get_mean_vector(model, sentence)
      vectors.append(vec)
  return vectors

In [9]:
def get_max_len_sentence(series):
  res = series.str.split().str.len().max()

  print(f"The maximum length in words are : {res}") 

In [10]:
data = pd.read_csv("/content/drive/MyDrive/hesaplamalı_anlambilim_ödev/preprocess_movie_sentiment.csv")

In [12]:
data.dropna(inplace=True)
data.reset_index(drop=True,inplace=True)

x=data.comment
y=data.sentiment.values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [13]:
corpus = labelize_tweets_ug(x, 'all')

In [14]:
corpus_train = pd.DataFrame(x_train)['comment'].apply(lambda x: x.split())
corpus_test = pd.DataFrame(x_test)['comment'].apply(lambda x: x.split())

In [22]:
# min_count = corpusta kelimenin en az bulunma sayısı eğer kelime bu kadardan az geçiyorsa anlam ifade etmediği varsayılır(default=5)
# vector_size = kelimelerin ifade edileceği vektörün boyut sayısı
# window = current ve predicted word arasındaki maksimum mesafe
# sg = 0 cbow, 1 skip-gram
# negative = eğer sıfırdan büyük olursa negative sampling kullanılır 5-20 arasında olmalı
# alpha = başlangıç learning rate
# min_alpha = eğitim aşamasında learning rate linear olarak bunu düşer
# epoch = iterasyon sayısı

# sg=0 epoch=25 negative=5 için (sg karar vermek için ilk deneme ikisi için yapılacak)
# alpha 0.5 0.01 0.05 0.1 her biri window = 3, vector_size = 150 kullanılacak 
# window 3 5 7 her biri alpha = 0.05, vector_size = 150 kullanılacak 
# vector_size 25 50 150 200 her biri  için window = 3, alpha = 0.05

In [16]:
model_1 = train_word2vec(corpus=corpus, 
                         n_epoch=25, 
                         name_corpus="movie", 
                         sg=0, 
                         negative=5,
                         alpha = 0.5,
                         min_alpha = 0.065,
                         window = 3,
                         vector_size=150,
                         min_count=2)

100%|██████████| 82598/82598 [00:00<00:00, 2008563.92it/s]


In [17]:
vectors_train = get_vectors(model=model_1,
                            corpus=corpus_train)
vectors_test = get_vectors(model=model_1,
                            corpus=corpus_test)

X_train = np.array(vectors_train)
X_train = np.vstack(X_train)
X_test = np.array(vectors_test)
X_test = np.vstack(X_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [18]:
classification_report(x_train=X_train,
                      x_test=X_test,
                      y_train=y_train,
                      y_test=y_test)

LogisticRegression:
accuracy: 0.662953995157385
precision: 0.6617183760481924
recall: 0.6597227901092813
              precision    recall  f1-score   support

           0       0.65      0.61      0.63      7761
           1       0.67      0.71      0.69      8759

    accuracy                           0.66     16520
   macro avg       0.66      0.66      0.66     16520
weighted avg       0.66      0.66      0.66     16520

RandomForest:
accuracy: 0.6245157384987894
precision: 0.6226658114985069
recall: 0.6215070937251884
              precision    recall  f1-score   support

           0       0.61      0.57      0.59      7761
           1       0.64      0.67      0.65      8759

    accuracy                           0.62     16520
   macro avg       0.62      0.62      0.62     16520
weighted avg       0.62      0.62      0.62     16520



In [19]:
# deneme 2
model_2 = train_word2vec(corpus=corpus, 
                         n_epoch=25, 
                         name_corpus="movie", 
                         sg=1, 
                         negative=5,
                         alpha = 0.5,
                         min_alpha = 0.065,
                         window = 3,
                         vector_size=150,
                         min_count=2)

100%|██████████| 82598/82598 [00:00<00:00, 1985017.43it/s]


In [20]:
vectors_train = get_vectors(model=model_2,
                            corpus=corpus_train)
vectors_test = get_vectors(model=model_2,
                            corpus=corpus_test)

X_train = np.array(vectors_train)
X_train = np.vstack(X_train)
X_test = np.array(vectors_test)
X_test = np.vstack(X_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [21]:
classification_report(x_train=X_train,
                      x_test=X_test,
                      y_train=y_train,
                      y_test=y_test)

LogisticRegression:
accuracy: 0.6463680387409201
precision: 0.6448700988464475
recall: 0.643046718276733
              precision    recall  f1-score   support

           0       0.63      0.59      0.61      7761
           1       0.66      0.70      0.68      8759

    accuracy                           0.65     16520
   macro avg       0.64      0.64      0.64     16520
weighted avg       0.65      0.65      0.65     16520

RandomForest:
accuracy: 0.6059927360774818
precision: 0.6038140832946602
recall: 0.6015655824269046
              precision    recall  f1-score   support

           0       0.59      0.53      0.56      7761
           1       0.62      0.67      0.64      8759

    accuracy                           0.61     16520
   macro avg       0.60      0.60      0.60     16520
weighted avg       0.60      0.61      0.60     16520



# yukarıdaki sonuçlara göre sg=0 -> cbow yöntemi için daha başarılı oldu onunla diğer sonuçlar alınacak

# alpha(learning rate) = 0.01

In [23]:
model = train_word2vec(corpus=corpus, 
                         n_epoch=25, 
                         name_corpus="movie", 
                         sg=0, 
                         negative=5,
                         alpha = 0.01,
                         min_alpha = 0.065,
                         window = 3,
                         vector_size=150,
                         min_count=2)

100%|██████████| 82598/82598 [00:00<00:00, 2089638.23it/s]


In [24]:
vectors_train = get_vectors(model=model,
                            corpus=corpus_train)
vectors_test = get_vectors(model=model,
                            corpus=corpus_test)

X_train = np.array(vectors_train)
X_train = np.vstack(X_train)
X_test = np.array(vectors_test)
X_test = np.vstack(X_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [25]:
classification_report(x_train=X_train,
                      x_test=X_test,
                      y_train=y_train,
                      y_test=y_test)

LogisticRegression:
accuracy: 0.7291767554479419
precision: 0.7286878348709327
recall: 0.726584509339476
              precision    recall  f1-score   support

           0       0.72      0.68      0.70      7761
           1       0.73      0.77      0.75      8759

    accuracy                           0.73     16520
   macro avg       0.73      0.73      0.73     16520
weighted avg       0.73      0.73      0.73     16520

RandomForest:
accuracy: 0.7110774818401937
precision: 0.7106883425801436
recall: 0.70793813358819
              precision    recall  f1-score   support

           0       0.71      0.66      0.68      7761
           1       0.71      0.76      0.74      8759

    accuracy                           0.71     16520
   macro avg       0.71      0.71      0.71     16520
weighted avg       0.71      0.71      0.71     16520



# learning rate 0.05

In [26]:
model = train_word2vec(corpus=corpus, 
                         n_epoch=25, 
                         name_corpus="movie", 
                         sg=0, 
                         negative=5,
                         alpha = 0.05,
                         min_alpha = 0.065,
                         window = 3,
                         vector_size=150,
                         min_count=2)

100%|██████████| 82598/82598 [00:00<00:00, 2009892.33it/s]


In [27]:
vectors_train = get_vectors(model=model,
                            corpus=corpus_train)
vectors_test = get_vectors(model=model,
                            corpus=corpus_test)

X_train = np.array(vectors_train)
X_train = np.vstack(X_train)
X_test = np.array(vectors_test)
X_test = np.vstack(X_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [28]:
classification_report(x_train=X_train,
                      x_test=X_test,
                      y_train=y_train,
                      y_test=y_test)

LogisticRegression:
accuracy: 0.7398305084745763
precision: 0.7391997939211934
recall: 0.7376810163445704
              precision    recall  f1-score   support

           0       0.73      0.70      0.72      7761
           1       0.75      0.77      0.76      8759

    accuracy                           0.74     16520
   macro avg       0.74      0.74      0.74     16520
weighted avg       0.74      0.74      0.74     16520

RandomForest:
accuracy: 0.722457627118644
precision: 0.7218741567567772
recall: 0.7198517801168571
              precision    recall  f1-score   support

           0       0.72      0.68      0.70      7761
           1       0.73      0.76      0.74      8759

    accuracy                           0.72     16520
   macro avg       0.72      0.72      0.72     16520
weighted avg       0.72      0.72      0.72     16520



# learning rate 0.1

In [29]:
model = train_word2vec(corpus=corpus, 
                         n_epoch=25, 
                         name_corpus="movie", 
                         sg=0, 
                         negative=5,
                         alpha = 0.1,
                         min_alpha = 0.065,
                         window = 3,
                         vector_size=150,
                         min_count=2)

100%|██████████| 82598/82598 [00:00<00:00, 1865224.06it/s]


In [30]:
vectors_train = get_vectors(model=model,
                            corpus=corpus_train)
vectors_test = get_vectors(model=model,
                            corpus=corpus_test)

X_train = np.array(vectors_train)
X_train = np.vstack(X_train)
X_test = np.array(vectors_test)
X_test = np.vstack(X_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [31]:
classification_report(x_train=X_train,
                      x_test=X_test,
                      y_train=y_train,
                      y_test=y_test)

LogisticRegression:
accuracy: 0.7328087167070217
precision: 0.7319889537068984
recall: 0.7308830827772723
              precision    recall  f1-score   support

           0       0.72      0.70      0.71      7761
           1       0.74      0.76      0.75      8759

    accuracy                           0.73     16520
   macro avg       0.73      0.73      0.73     16520
weighted avg       0.73      0.73      0.73     16520

RandomForest:
accuracy: 0.7130750605326877
precision: 0.7128433291807558
recall: 0.7097998886384816
              precision    recall  f1-score   support

           0       0.71      0.66      0.68      7761
           1       0.71      0.76      0.74      8759

    accuracy                           0.71     16520
   macro avg       0.71      0.71      0.71     16520
weighted avg       0.71      0.71      0.71     16520



# window = 3

In [32]:
model = train_word2vec(corpus=corpus, 
                         n_epoch=25, 
                         name_corpus="movie", 
                         sg=0, 
                         negative=5,
                         alpha = 0.05,
                         min_alpha = 0.065,
                         window = 3,
                         vector_size=150,
                         min_count=2)

100%|██████████| 82598/82598 [00:00<00:00, 2035948.81it/s]


In [33]:
vectors_train = get_vectors(model=model,
                            corpus=corpus_train)
vectors_test = get_vectors(model=model,
                            corpus=corpus_test)

X_train = np.array(vectors_train)
X_train = np.vstack(X_train)
X_test = np.array(vectors_test)
X_test = np.vstack(X_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [34]:
classification_report(x_train=X_train,
                      x_test=X_test,
                      y_train=y_train,
                      y_test=y_test)

LogisticRegression:
accuracy: 0.7395883777239709
precision: 0.7389859341046865
recall: 0.7373866148668349
              precision    recall  f1-score   support

           0       0.73      0.70      0.72      7761
           1       0.74      0.77      0.76      8759

    accuracy                           0.74     16520
   macro avg       0.74      0.74      0.74     16520
weighted avg       0.74      0.74      0.74     16520

RandomForest:
accuracy: 0.7204600484261501
precision: 0.7202135129479756
recall: 0.7173734192433121
              precision    recall  f1-score   support

           0       0.72      0.67      0.69      7761
           1       0.72      0.77      0.74      8759

    accuracy                           0.72     16520
   macro avg       0.72      0.72      0.72     16520
weighted avg       0.72      0.72      0.72     16520



# window size = 5

In [35]:
model = train_word2vec(corpus=corpus, 
                         n_epoch=25, 
                         name_corpus="movie", 
                         sg=0, 
                         negative=5,
                         alpha = 0.05,
                         min_alpha = 0.065,
                         window = 5,
                         vector_size=150,
                         min_count=2)

100%|██████████| 82598/82598 [00:00<00:00, 2009157.99it/s]


In [36]:
vectors_train = get_vectors(model=model,
                            corpus=corpus_train)
vectors_test = get_vectors(model=model,
                            corpus=corpus_test)

X_train = np.array(vectors_train)
X_train = np.vstack(X_train)
X_test = np.array(vectors_test)
X_test = np.vstack(X_test)

classification_report(x_train=X_train,
                      x_test=X_test,
                      y_train=y_train,
                      y_test=y_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


LogisticRegression:
accuracy: 0.7414043583535109
precision: 0.7408347136112996
recall: 0.7391872256737742
              precision    recall  f1-score   support

           0       0.74      0.70      0.72      7761
           1       0.75      0.78      0.76      8759

    accuracy                           0.74     16520
   macro avg       0.74      0.74      0.74     16520
weighted avg       0.74      0.74      0.74     16520

RandomForest:
accuracy: 0.7269975786924939
precision: 0.7265949550601012
recall: 0.7242431989514817
              precision    recall  f1-score   support

           0       0.72      0.68      0.70      7761
           1       0.73      0.77      0.75      8759

    accuracy                           0.73     16520
   macro avg       0.73      0.72      0.72     16520
weighted avg       0.73      0.73      0.73     16520



# window size = 7

In [37]:
model = train_word2vec(corpus=corpus, 
                         n_epoch=25, 
                         name_corpus="movie", 
                         sg=0, 
                         negative=5,
                         alpha = 0.05,
                         min_alpha = 0.065,
                         window = 7,
                         vector_size=150,
                         min_count=2)


vectors_train = get_vectors(model=model,
                            corpus=corpus_train)
vectors_test = get_vectors(model=model,
                            corpus=corpus_test)

X_train = np.array(vectors_train)
X_train = np.vstack(X_train)
X_test = np.array(vectors_test)
X_test = np.vstack(X_test)

classification_report(x_train=X_train,
                      x_test=X_test,
                      y_train=y_train,
                      y_test=y_test)

100%|██████████| 82598/82598 [00:00<00:00, 2091833.65it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


LogisticRegression:
accuracy: 0.7463075060532688
precision: 0.7456951923186192
recall: 0.7442588144542372
              precision    recall  f1-score   support

           0       0.74      0.71      0.72      7761
           1       0.75      0.78      0.76      8759

    accuracy                           0.75     16520
   macro avg       0.75      0.74      0.74     16520
weighted avg       0.75      0.75      0.75     16520

RandomForest:
accuracy: 0.7291767554479419
precision: 0.7287146313762283
recall: 0.7265478066119015
              precision    recall  f1-score   support

           0       0.72      0.68      0.70      7761
           1       0.73      0.77      0.75      8759

    accuracy                           0.73     16520
   macro avg       0.73      0.73      0.73     16520
weighted avg       0.73      0.73      0.73     16520



# vector size = 25


In [38]:
model = train_word2vec(corpus=corpus, 
                         n_epoch=25, 
                         name_corpus="movie", 
                         sg=0, 
                         negative=5,
                         alpha = 0.05,
                         min_alpha = 0.065,
                         window = 3,
                         vector_size=25,
                         min_count=2)


vectors_train = get_vectors(model=model,
                            corpus=corpus_train)
vectors_test = get_vectors(model=model,
                            corpus=corpus_test)

X_train = np.array(vectors_train)
X_train = np.vstack(X_train)
X_test = np.array(vectors_test)
X_test = np.vstack(X_test)

classification_report(x_train=X_train,
                      x_test=X_test,
                      y_train=y_train,
                      y_test=y_test)

100%|██████████| 82598/82598 [00:00<00:00, 2063943.20it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


LogisticRegression:
accuracy: 0.7207021791767555
precision: 0.7198313647010177
recall: 0.7185927294559278
              precision    recall  f1-score   support

           0       0.71      0.68      0.70      7761
           1       0.73      0.75      0.74      8759

    accuracy                           0.72     16520
   macro avg       0.72      0.72      0.72     16520
weighted avg       0.72      0.72      0.72     16520

RandomForest:
accuracy: 0.7177966101694915
precision: 0.7172028932941477
recall: 0.7150966144506744
              precision    recall  f1-score   support

           0       0.71      0.67      0.69      7761
           1       0.72      0.76      0.74      8759

    accuracy                           0.72     16520
   macro avg       0.72      0.72      0.72     16520
weighted avg       0.72      0.72      0.72     16520



# vector  size = 50

In [39]:
model = train_word2vec(corpus=corpus, 
                         n_epoch=25, 
                         name_corpus="movie", 
                         sg=0, 
                         negative=5,
                         alpha = 0.05,
                         min_alpha = 0.065,
                         window = 3,
                         vector_size=50,
                         min_count=2)


vectors_train = get_vectors(model=model,
                            corpus=corpus_train)
vectors_test = get_vectors(model=model,
                            corpus=corpus_test)

X_train = np.array(vectors_train)
X_train = np.vstack(X_train)
X_test = np.array(vectors_test)
X_test = np.vstack(X_test)

classification_report(x_train=X_train,
                      x_test=X_test,
                      y_train=y_train,
                      y_test=y_test)

100%|██████████| 82598/82598 [00:00<00:00, 1968236.53it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


LogisticRegression:
accuracy: 0.7325665859564164
precision: 0.7317894363189625
recall: 0.7305372974809321
              precision    recall  f1-score   support

           0       0.72      0.70      0.71      7761
           1       0.74      0.76      0.75      8759

    accuracy                           0.73     16520
   macro avg       0.73      0.73      0.73     16520
weighted avg       0.73      0.73      0.73     16520

RandomForest:
accuracy: 0.7275423728813559
precision: 0.7270774169291097
recall: 0.7248817455034635
              precision    recall  f1-score   support

           0       0.72      0.68      0.70      7761
           1       0.73      0.77      0.75      8759

    accuracy                           0.73     16520
   macro avg       0.73      0.72      0.73     16520
weighted avg       0.73      0.73      0.73     16520



# vector size = 100

In [40]:
model = train_word2vec(corpus=corpus, 
                         n_epoch=25, 
                         name_corpus="movie", 
                         sg=0, 
                         negative=5,
                         alpha = 0.05,
                         min_alpha = 0.065,
                         window = 3,
                         vector_size=100,
                         min_count=2)


vectors_train = get_vectors(model=model,
                            corpus=corpus_train)
vectors_test = get_vectors(model=model,
                            corpus=corpus_test)

X_train = np.array(vectors_train)
X_train = np.vstack(X_train)
X_test = np.array(vectors_test)
X_test = np.vstack(X_test)

classification_report(x_train=X_train,
                      x_test=X_test,
                      y_train=y_train,
                      y_test=y_test)

100%|██████████| 82598/82598 [00:00<00:00, 2072065.85it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


LogisticRegression:
accuracy: 0.7372881355932204
precision: 0.7367068604923366
recall: 0.735004541649939
              precision    recall  f1-score   support

           0       0.73      0.70      0.71      7761
           1       0.74      0.77      0.76      8759

    accuracy                           0.74     16520
   macro avg       0.74      0.74      0.74     16520
weighted avg       0.74      0.74      0.74     16520

RandomForest:
accuracy: 0.7230629539951574
precision: 0.7225172085068203
recall: 0.7204152809915956
              precision    recall  f1-score   support

           0       0.72      0.68      0.70      7761
           1       0.73      0.76      0.75      8759

    accuracy                           0.72     16520
   macro avg       0.72      0.72      0.72     16520
weighted avg       0.72      0.72      0.72     16520



# vector size = 150

In [41]:
model = train_word2vec(corpus=corpus, 
                         n_epoch=25, 
                         name_corpus="movie", 
                         sg=0, 
                         negative=5,
                         alpha = 0.05,
                         min_alpha = 0.065,
                         window = 3,
                         vector_size=150,
                         min_count=2)


vectors_train = get_vectors(model=model,
                            corpus=corpus_train)
vectors_test = get_vectors(model=model,
                            corpus=corpus_test)

X_train = np.array(vectors_train)
X_train = np.vstack(X_train)
X_test = np.array(vectors_test)
X_test = np.vstack(X_test)

classification_report(x_train=X_train,
                      x_test=X_test,
                      y_train=y_train,
                      y_test=y_test)

100%|██████████| 82598/82598 [00:00<00:00, 2124193.69it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


LogisticRegression:
accuracy: 0.7415859564164649
precision: 0.7410347734077674
recall: 0.7393437970088204
              precision    recall  f1-score   support

           0       0.74      0.70      0.72      7761
           1       0.75      0.78      0.76      8759

    accuracy                           0.74     16520
   macro avg       0.74      0.74      0.74     16520
weighted avg       0.74      0.74      0.74     16520

RandomForest:
accuracy: 0.7216707021791767
precision: 0.7212199924788967
recall: 0.7188601077230203
              precision    recall  f1-score   support

           0       0.72      0.67      0.69      7761
           1       0.72      0.77      0.74      8759

    accuracy                           0.72     16520
   macro avg       0.72      0.72      0.72     16520
weighted avg       0.72      0.72      0.72     16520



# vector size = 200

In [42]:
model = train_word2vec(corpus=corpus, 
                         n_epoch=25, 
                         name_corpus="movie", 
                         sg=0, 
                         negative=5,
                         alpha = 0.05,
                         min_alpha = 0.065,
                         window = 3,
                         vector_size=200,
                         min_count=2)


vectors_train = get_vectors(model=model,
                            corpus=corpus_train)
vectors_test = get_vectors(model=model,
                            corpus=corpus_test)

X_train = np.array(vectors_train)
X_train = np.vstack(X_train)
X_test = np.array(vectors_test)
X_test = np.vstack(X_test)

classification_report(x_train=X_train,
                      x_test=X_test,
                      y_train=y_train,
                      y_test=y_test)

100%|██████████| 82598/82598 [00:00<00:00, 2117170.77it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


LogisticRegression:
accuracy: 0.7407990314769975
precision: 0.7403059721672947
recall: 0.7384548922521925
              precision    recall  f1-score   support

           0       0.74      0.70      0.72      7761
           1       0.74      0.78      0.76      8759

    accuracy                           0.74     16520
   macro avg       0.74      0.74      0.74     16520
weighted avg       0.74      0.74      0.74     16520

RandomForest:
accuracy: 0.7200968523002421
precision: 0.7196510446690256
recall: 0.7172437902110929
              precision    recall  f1-score   support

           0       0.72      0.67      0.69      7761
           1       0.72      0.76      0.74      8759

    accuracy                           0.72     16520
   macro avg       0.72      0.72      0.72     16520
weighted avg       0.72      0.72      0.72     16520



# en iyiler sq=0, alpha=0.1, window_size=7, vector_size=200

In [43]:
model = train_word2vec(corpus=corpus, 
                         n_epoch=25, 
                         name_corpus="movie", 
                         sg=0, 
                         negative=5,
                         alpha = 0.1,
                         min_alpha = 0.065,
                         window = 7,
                         vector_size=200,
                         min_count=2)


vectors_train = get_vectors(model=model,
                            corpus=corpus_train)
vectors_test = get_vectors(model=model,
                            corpus=corpus_test)

X_train = np.array(vectors_train)
X_train = np.vstack(X_train)
X_test = np.array(vectors_test)
X_test = np.vstack(X_test)

classification_report(x_train=X_train,
                      x_test=X_test,
                      y_train=y_train,
                      y_test=y_test)

100%|██████████| 82598/82598 [00:00<00:00, 1946560.89it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


LogisticRegression:
accuracy: 0.7354721549636803
precision: 0.7351110127728779
recall: 0.7328442441127685
              precision    recall  f1-score   support

           0       0.73      0.69      0.71      7761
           1       0.74      0.78      0.76      8759

    accuracy                           0.74     16520
   macro avg       0.74      0.73      0.73     16520
weighted avg       0.74      0.74      0.73     16520

RandomForest:
accuracy: 0.7096246973365617
precision: 0.70977045191627
recall: 0.7058854434466941
              precision    recall  f1-score   support

           0       0.71      0.64      0.68      7761
           1       0.71      0.77      0.74      8759

    accuracy                           0.71     16520
   macro avg       0.71      0.71      0.71     16520
weighted avg       0.71      0.71      0.71     16520

